In [5]:
# Since the file is on a subfolder of the git we need to add the main folder to the path

import sys
import os 
path = os.getcwd()
path_root = path[:-7]
sys.path.append(str(path_root))
from src.PAS import Patient, disturbances, metrics

import numpy as np
import pandas as pd
from Controller import GPC
from pyswarm import pso
import casadi as cas
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.io import output_notebook
output_notebook() 

Loading BokehJS ...

In [2]:
#Patient table:
#index, Age, H[cm], W[kg], Gender, Ce50p, Ce50r, γ, β, E0, Emax
Patient_table = [[1,  40, 163, 54, 0, 6.33, 12.5, 2.24, 2.00, 98.8, 94.1],
                 [2,  36, 163, 50, 0, 6.76, 12.7, 4.29, 1.50, 98.6, 86],
                 [3,  28, 164, 52, 0, 8.44, 7.1,  4.10, 1.00, 91.2, 80.7],
                 [4,  50, 163, 83, 0, 6.44, 11.1, 2.18, 1.30, 95.9, 102],
                 [5,  28, 164, 60, 1, 4.93, 12.5, 2.46, 1.20, 94.7, 85.3],
                 [6,  43, 163, 59, 0, 12.0, 12.7, 2.42, 1.30, 90.2, 147],
                 [7,  37, 187, 75, 1, 8.02, 10.5, 2.10, 0.80, 92.0, 104],
                 [8,  38, 174, 80, 0, 6.56, 9.9,  4.12, 1.00, 95.5, 76.4],
                 [9,  41, 170, 70, 0, 6.15, 11.6, 6.89, 1.70, 89.2, 63.8],
                 [10, 37, 167, 58, 0, 13.7, 16.7, 3.65, 1.90, 83.1, 151],
                 [11, 42, 179, 78, 1, 4.82, 14.0, 1.85, 1.20, 91.8, 77.9],
                 [12, 34, 172, 58, 0, 4.95, 8.8,  1.84, 0.90, 96.2, 90.8],
                 [13, 38, 169, 65, 0, 7.42, 10.5, 3.00, 1.00, 93.1, 96.58]]


## Define simulation function

In [3]:
def simu(Patient_info: list,style: str, MPC_param: list, random_PK: bool = False, random_PD: bool = False):
    ''' This function perform a closed-loop Propofol-Remifentanil anesthesia simulation with a PID controller.
    
    Inputs: - Patient_info: list of patient informations, Patient_info = [Age, H[cm], W[kg], Gender, Ce50p, Ce50r, γ, β, E0, Emax]
            - style: either 'induction' or 'maintenance' to describe the phase to simulate
            - MPC_param: parameter of the PID controller P = [N, Q, R]
            - random: bool to add uncertainty to simulate intra-patient variability in the patient model
    
    Outputs:- IAE: Integrated Absolute Error, performance index of the function
            - data: list of the signals during the simulation data = [BIS, MAP, CO, up, ur]
    '''
    
    age = Patient_info[0]
    height = Patient_info[1]
    weight = Patient_info[2]
    gender = Patient_info[3]
    Ce50p = Patient_info[4]
    Ce50r = Patient_info[5]
    gamma = Patient_info[6]
    beta = Patient_info[7]
    E0 = Patient_info[8]
    Emax = Patient_info[9]
    
    ts = 5
    
    BIS_param = [Ce50p, Ce50r, gamma, beta, E0, Emax]
    George = Patient.Patient(age, height, weight, gender, BIS_param = BIS_param,
                             Random_PK = random_PK, Random_PD = random_PD, Te = ts)#, model_propo = 'Eleveld', model_remi = 'Eleveld')

    #Nominal parameters
    George_nominal = Patient.Patient(age, height, weight, gender, BIS_param = BIS_param, Te = ts)
    BIS_param_nominal = George_nominal.BisPD.BIS_param
    
    Ap = George_nominal.PropoPK.A_d
    Ar = George_nominal.RemiPK.A_d
    Bp = George_nominal.PropoPK.B_d
    Br = George_nominal.RemiPK.B_d

    #init controller
    N_mpc = MPC_param[0]
    Nu_mpc = MPC_param[1]
    lambda_param = MPC_param[2]

    BIS_cible = 50
    up_max = 6.67
    ur_max = 16.67
    K = 2
    MPC_controller = GPC(Ap, Bp, Ar, Br, BIS_param = BIS_param_nominal, ts = ts, N = N_mpc, Nu = Nu_mpc,
                          lambda_param = lambda_param, umax = min(up_max, ur_max/K))




    if style=='induction':
        N_simu = int(30/ts)*60
        BIS = np.zeros(N_simu)
        MAP = np.zeros(N_simu)
        CO = np.zeros(N_simu)
        Up = np.zeros(N_simu)
        Ur = np.zeros(N_simu)
        Xp = np.zeros((4, N_simu))
        Xr = np.zeros((4, N_simu))
        uP = 0
        uR = 0
        for i in range(N_simu):

            Dist = disturbances.compute_disturbances(i*ts, 'null')
            Bis, Co, Map = George.one_step(uP, uR, Dist = Dist, noise = False)
            Xp[:,i] =  George.PropoPK.x.T[0]       
            Xr[:,i] =  George.RemiPK.x.T[0]          

            BIS[i] = min(100,Bis)
            MAP[i] = Map[0,0]
            CO[i] = Co[0,0]
            Up[i] = uP
            Ur[i] = uR
            #estimation
            uP, uR = MPC_controller.one_step(BIS[i], BIS_cible)         

    elif style=='total':
        N_simu = int(60/ts)*60
        BIS = np.zeros(N_simu)
        BIS_EKF = np.zeros(N_simu)
        MAP = np.zeros(N_simu)
        CO = np.zeros(N_simu)
        Up = np.zeros(N_simu)
        Ur = np.zeros(N_simu)
        Xp = np.zeros((4, N_simu))
        Xr = np.zeros((4, N_simu))
        Xp_EKF = np.zeros((4, N_simu))
        Xr_EKF = np.zeros((4, N_simu))
        uP = 1
        uR = 1
        for i in range(N_simu):
            # if i == 100:
            #     print("break")

            Dist = disturbances.compute_disturbances(i*ts, 'realistic')
            Bis, Co, Map = George.one_step(uP, uR, Dist = Dist, noise = True)
            Xp[:,i] =  George.PropoPK.x.T[0]       
            Xr[:,i] =  George.RemiPK.x.T[0]          

            BIS[i] = min(100,Bis)
            MAP[i] = Map[0,0]
            CO[i] = Co[0,0]
            Up[i] = uP
            Ur[i] = uR
            #estimation
            uP, uR = MPC_controller.one_step(BIS[i], BIS_cible) 
      
    IAE = np.sum(np.abs(BIS - BIS_cible))
    return IAE, [BIS, MAP, CO, Up, Ur]

## Perform table population simulation

Test all the drug ratio on all the patient in the table


In [4]:
phase = 'induction'
MPC_param = [36, 34, 10]


IAE_list = []
TT_list = []
p1 = figure(plot_width=900, plot_height=300)
p2 = figure(plot_width=900, plot_height=300)
p3 = figure(plot_width=900, plot_height=300)
for i in range(1,14):
    Patient_info = Patient_table[i-1][1:]
    IAE, data = simu(Patient_info, phase, MPC_param)
    p1.line(np.arange(0,len(data[0]))/60, data[0])
    p2.line(np.arange(0,len(data[0]))/60, data[1], legend_label='MAP (mmgh)')
    p2.line(np.arange(0,len(data[0]))/60, data[2]*10, legend_label='CO (cL/min)', line_color="#f46d43")
    p3.line(np.arange(0,len(data[3]))/60, data[3], line_color="#006d43", legend_label='propofol (mg/min)')
    p3.line(np.arange(0,len(data[4]))/60, data[4], line_color="#f46d43", legend_label='remifentanil (ng/min)')
    TT, BIS_NADIR, ST10, ST20, US = metrics.compute_control_metrics(data[0], Te = 1, phase = phase)
    TT_list.append(TT)
    IAE_list.append(IAE)
p1.title.text = 'BIS'
p3.title.text = 'Infusion rates'
p1.xaxis.axis_label = 'Time (min)'
p2.xaxis.axis_label = 'Time (min)'
p3.xaxis.axis_label = 'Time (min)'
grid = row(column(p3,p1,p2))

show(grid)

print("Mean IAE : " + str(np.mean(IAE_list)))
print("Mean TT : " + str(np.mean(TT_list)))
print("Min TT : " + str(np.min(TT_list)))
print("Max TT : " + str(np.max(TT_list)))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



Mean IAE : 1707.0366321858678
Mean TT : 0.23333333333333336
Min TT : 0.2
Max TT : 0.25


## Inter Patient variability test

In [6]:
#%% #Simulation parameter
phase = 'induction'
Number_of_patient = 50
MPC_param = [36, 34, 10]

IAE_list = []
TT_list = []
p1 = figure(plot_width=900, plot_height=300)
p2 = figure(plot_width=900, plot_height=300)
p3 = figure(plot_width=900, plot_height=300)

for i in range(Number_of_patient):
    #Generate random patient information with uniform distribution
    age = 30 #np.random.randint(low=18,high=70)
    height = 175#np.random.randint(low=150,high=190)
    weight = 75#np.random.randint(low=50,high=100)
    gender = 1#np.random.randint(low=0,high=1)

    Patient_info = [age, height, weight, gender] + [None]*6
    IAE, data = simu(Patient_info, phase, MPC_param, random_PD = True)
    p1.line(np.arange(0,len(data[0]))*5/60, data[0])
    # p1.line(np.arange(0,len(data[0]))*5/60, data[5], legend_label='internal target', line_color="#f46d43")
    p2.line(np.arange(0,len(data[0]))*5/60, data[1], legend_label='MAP (mmgh)')
    p2.line(np.arange(0,len(data[0]))*5/60, data[2]*10, legend_label='CO (cL/min)', line_color="#f46d43")
    p3.line(np.arange(0,len(data[3]))*5/60, data[3], line_color="#006d43", legend_label='propofol (mg/min)')
    p3.line(np.arange(0,len(data[4]))*5/60, data[4], line_color="#f46d43", legend_label='remifentanil (ng/min)')
    TT, BIS_NADIR, ST10, ST20, US = metrics.compute_control_metrics(data[0], Te = 5, phase = phase)
    TT_list.append(TT)
    IAE_list.append(IAE)
    
p1.title.text = 'BIS'
p3.title.text = 'Infusion rates'
p3.xaxis.axis_label = 'Time (min)'
grid = row(column(p3,p1,p2))

show(grid)

print("Mean IAE : " + str(np.mean(IAE_list)))
print("Mean TT : " + str(np.mean(TT_list)))
print("Min TT : " + str(np.min(TT_list)))
print("Max TT : " + str(np.max(TT_list)))


/home/aubouinb/ownCloud/Anesthesie/Science/Bob/Code/Python_Anesthesia_Simulator/example/Controller.py:492: RuntimeWarning: invalid value encountered in double_scalars
  temp = (max(0,E0-Bis)/(Emax-E0+Bis))**(1/gamma)


LinAlgError: Array must not contain infs or NaNs

## Intra-patient variability

In [ ]:
#Simulation parameter
phase = 'induction'
Number_of_patient = 50
MPC_param = [36, 34, 10]

IAE_list = []
TT_list = []
p1 = figure(plot_width=900, plot_height=300)
p2 = figure(plot_width=900, plot_height=300)
p3 = figure(plot_width=900, plot_height=300)

for i in range(Number_of_patient):
    print(i)
    #Use patient information from average table patient 
    Patient_info = Patient_table[12][1:]
    Patient_info[4:] = [None]*6

    IAE, data = simu(Patient_info, phase, MPC_param, random_PK = True, random_PD = True)
    p1.line(np.arange(0,len(data[0]))*5/60, data[0])
    p2.line(np.arange(0,len(data[0]))*5/60, data[1], legend_label='MAP (mmgh)')
    p2.line(np.arange(0,len(data[0]))*5/60, data[2]*10, legend_label='CO (cL/min)', line_color="#f46d43")
    p3.line(np.arange(0,len(data[3]))*5/60, data[3], line_color="#006d43", legend_label='propofol (mg/min)')
    p3.line(np.arange(0,len(data[4]))*5/60, data[4], line_color="#f46d43", legend_label='remifentanil (ng/min)')
    TT, BIS_NADIR, ST10, ST20, US = metrics.compute_control_metrics(data[0], Te = 1, phase = phase)
    TT_list.append(TT)
    IAE_list.append(IAE)
    
p1.title.text = 'BIS'
p3.title.text = 'Infusion rates'
p3.xaxis.axis_label = 'Time (min)'
grid = row(column(p3,p1,p2))

show(grid)

print("Mean IAE : " + str(np.mean(IAE_list)))
print("Mean TT : " + str(np.mean(TT_list)))
print("Min TT : " + str(np.min(TT_list)))
print("Max TT : " + str(np.max(TT_list)))